In [4]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import os

In [5]:
weighted_adj_matrix_path = os.path.join('distances23.csv')
W = pd.read_csv(weighted_adj_matrix_path).drop('station_id', axis=1)


In [52]:
all_df = pd.read_pickle(r'C:\Users\srush\bike share model\cleaned&merged_data\All_Rides.pkl')

In [53]:
# Filtering JC Stations for analysis
jc_stations = ['JC102', 'JC096', 'JC0003', 'JC099', 'JC023', 'JC081', 'JC075', 'JC078', 'JC063', 'JC080', 'JC072', 'JC013', 'JC002', 'JC076', 'JC032', 'JC027', 'JC009', 'JC105', 'JC053', 'JC093', 'JC020', 'JC094', 'JC056', 'JC103', 'JC038', 'JC006', 'JC104', 'JC008', 'JC0084', 'JC065', 'JC024', 'JC059', 'JC0011', 'JC0035', 'JC005', 'JC082', 'JC074', 'JC055', 'JC022', 'JC106', 'JC014', 'JC098', 'JC066', 'JC052', 'JC051', 'JC077', 'JC0095', 'JC019', 'JC018', 'JC0034', 'JC057']

df = all_df[(all_df['start_station_id'].isin(jc_stations)) & (all_df['end_station_id'].isin(jc_stations))]

In [54]:
#converting to date and time type
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])

C:\Users\srush\AppData\Local\Temp\ipykernel_14520\1621484384.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['started_at'] = pd.to_datetime(df['started_at'])
C:\Users\srush\AppData\Local\Temp\ipykernel_14520\1621484384.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ended_at'] = pd.to_datetime(df['ended_at'])


In [55]:
# Define start and end dates
start_date = '2023-03-01 00:00:00'
end_date = '2023-05-31 23:55:00'

# Create a mask for the 'started_at' column
start_date_mask = df['started_at'].between(start_date, end_date)

# Apply the mask to the DataFrame
df = df[start_date_mask]

# Create a mask for the 'ended_at' column
end_date_mask = df['ended_at'].between(start_date, end_date)

# Apply the mask to the DataFrame
df = df[end_date_mask]


In [56]:
max_sdatetime = df['started_at'].max()
min_sdatetime = df['started_at'].min()

print("Maximum start datetime:", max_sdatetime)
print("Minimum start datetime:", min_sdatetime)

max_edatetime = df['ended_at'].max()
min_edatetime = df['ended_at'].min()

print("Maximum end datetime:", max_edatetime)
print("Minimum end datetime:", min_edatetime)

Maximum start datetime: 2023-05-31 23:39:40
Minimum start datetime: 2023-03-01 00:25:35
Maximum end datetime: 2023-05-31 23:44:45
Minimum end datetime: 2023-03-01 00:31:59


In [57]:
common_start_time = max(min_sdatetime, min_edatetime)
common_end_time = min(max_sdatetime, max_edatetime)

df = df[(df['started_at'] >= common_start_time) & (df['started_at'] <= common_end_time)]
df = df[(df['ended_at'] >= common_start_time) & (df['ended_at'] <= common_end_time)]

In [58]:
max_sdatetime = df['started_at'].max()
min_sdatetime = df['started_at'].min()

print("Maximum start datetime:", max_sdatetime)
print("Minimum start datetime:", min_sdatetime)

max_edatetime = df['ended_at'].max()
min_edatetime = df['ended_at'].min()

print("Maximum end datetime:", max_edatetime)
print("Minimum end datetime:", min_edatetime)

Maximum start datetime: 2023-05-31 23:19:50
Minimum start datetime: 2023-03-01 00:43:53
Maximum end datetime: 2023-05-31 23:29:28
Minimum end datetime: 2023-03-01 00:48:33


In [59]:
# All unique stations in Jersey City
unique_stations = set(df['start_station_name']).union(set(df['end_station_name']))
number_of_unique_stations = len(unique_stations)
number_of_unique_stations

41

In [120]:
# Create a mapping of station IDs to matrix indices
station_index = {station: idx for idx, station in enumerate(unique_stations)}

In [121]:
adjacency_matrix = np.zeros((number_of_unique_stations, number_of_unique_stations), dtype=int)

In [122]:
for _, row in df.iterrows():
    start_idx = station_index[row['start_station_name']]
    end_idx = station_index[row['end_station_name']]
    adjacency_matrix[start_idx, end_idx] += 1

In [123]:
np.save('adjacency_matrix.npy', adjacency_matrix)

In [60]:
# Count the number of rides for each start station
start_station_counts = df.groupby('start_station_name').size().sort_values(ascending=False)

# Count the number of rides for each end station
end_station_counts = df.groupby('end_station_name').size().sort_values(ascending=False)

In [61]:
df.shape

(58098, 12)

In [62]:
# Create in-data frame : stations and their incoming rides
in_df = df.drop(['started_at', 'start_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng','member_casual','start_station_name','end_station_name'], axis=1)
in_df['time'] = df['ended_at']
#in_df = in_df[(in_df['time'] >= '2017-01-01 00:21:00') & (in_df['time'] <= '2023-09-30 23:59:59')]
in_df['station_id'] = df['end_station_id']
in_df = in_df.drop(['ended_at', 'end_station_id'], axis=1)

# out-dataframe : stations with the rides leaving the station
out_df = df.drop(['ended_at', 'end_station_id','start_lat', 'start_lng', 'end_lat', 'end_lng','member_casual','start_station_name','end_station_name'], axis=1)
out_df['time'] = df['started_at']
#out_df = out_df[(out_df['time'] >= '2017-01-01 00:21:00') & (out_df['time'] <= '2023-09-30 23:59:59')]
out_df['station_id'] = df['start_station_id']
out_df = out_df.drop(['started_at', 'start_station_id'], axis=1)

In [63]:
in_df.tail()

,ride_id,time,station_id
1875342,2ED9BC6D822C18D9,2023-05-23 18:25:32,JC008
1875344,53D865CBD9BFC5F4,2023-05-07 08:06:04,JC008
1875345,ABAFB5950E7C79B7,2023-05-06 02:44:09,JC006
1875347,E266F73A92C04C92,2023-05-31 15:54:00,JC008
1875352,CABAC21C0F0140A5,2023-05-16 12:04:04,JC008


In [64]:
in_df.shape

(58098, 3)

In [65]:
out_df.tail()

,ride_id,time,station_id
1875342,2ED9BC6D822C18D9,2023-05-23 18:19:10,JC014
1875344,53D865CBD9BFC5F4,2023-05-07 07:56:04,JC014
1875345,ABAFB5950E7C79B7,2023-05-06 02:42:43,JC014
1875347,E266F73A92C04C92,2023-05-31 15:39:26,JC014
1875352,CABAC21C0F0140A5,2023-05-16 11:56:11,JC014


In [66]:
out_df.shape

(58098, 3)

In [67]:
#number of 5-minute intervals throughout the data
timestep = 5
date_range = pd.date_range(start='2023-03-01 00:00:00', end='2023-04-28 23:55:00', freq=f'{timestep}T')

# Calculate the length of the date range
num_intervals = len(date_range)
num_intervals

16992

In [22]:
#Out rides
out_timeseries = out_df.groupby([pd.Grouper(freq=str(timestep)+'T', key='time'), 'station_id'])
out_timeseries = out_timeseries['ride_id'].count().unstack('station_id', fill_value=0)

# Convert all column names to strings, then sort
out_timeseries.columns = out_timeseries.columns.map(str)
out_timeseries = out_timeseries.reindex(sorted(out_timeseries.columns), axis=1)
print('Out Timeseries length:', len(out_timeseries))
print(print(pd.Series(out_timeseries.values.ravel()).describe()))

Out Timeseries length: 11289
count    462849.000000
mean          0.074378
std           0.299946
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           7.000000
dtype: float64
None


In [23]:
out_timeseries.tail()

station_id,JC002,JC006,JC008,JC009,JC013,JC014,JC018,JC019,JC020,JC022,...,JC081,JC082,JC093,JC094,JC098,JC099,JC102,JC103,JC104,JC105
time,,,,,,,,,,,,,,,,,,,,,
2023-04-28 22:05:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-04-28 22:15:00,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2023-04-28 22:20:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-04-28 22:35:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-04-28 22:45:00,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
#In rides
in_timeseries = in_df.groupby([pd.Grouper(freq=str(timestep)+'T', key='time'), 'station_id'])
in_timeseries = in_timeseries['ride_id'].count().unstack('station_id', fill_value=0)
in_timeseries.columns = in_timeseries.columns.map(str)
in_timeseries = in_timeseries.reindex(sorted(in_timeseries.columns), axis=1)
print('\nIn Timeseries:', len(in_timeseries))
print(print(pd.Series(in_timeseries.values.ravel()).describe()))


In Timeseries: 18025
count    739025.000000
mean          0.078614
std           0.308873
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           9.000000
dtype: float64
None


In [69]:
in_timeseries.tail()

station_id,JC002,JC006,JC008,JC009,JC013,JC014,JC018,JC019,JC020,JC022,...,JC081,JC082,JC093,JC094,JC098,JC099,JC102,JC103,JC104,JC105
time,,,,,,,,,,,,,,,,,,,,,
2023-05-31 23:05:00,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-05-31 23:10:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-05-31 23:15:00,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-05-31 23:20:00,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-05-31 23:25:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
min_out_time = out_df['time'].min()
max_out_time = out_df['time'].max()
min_in_time = in_df['time'].min()
max_in_time = in_df['time'].max()

print("Out Timeseries range:", min_out_time, "to", max_out_time)
print("In Timeseries range:", min_in_time, "to", max_in_time)

Out Timeseries range: 2023-03-01 00:43:53 to 2023-05-31 23:19:50
In Timeseries range: 2023-03-01 00:48:33 to 2023-05-31 23:29:28


In [71]:
# Assuming min_out_time, max_out_time, min_in_time, and max_in_time are already defined
# Find the latest start time
common_start_time = max(min_out_time, min_in_time)

# Find the earliest end time
common_end_time = min(max_out_time, max_in_time)

# Now filter both dataframes to only include rows within the common time range
out_df_common = out_df[(out_df['time'] >= common_start_time) & (out_df['time'] <= common_end_time)]
in_df_common = in_df[(in_df['time'] >= common_start_time) & (in_df['time'] <= common_end_time)]

# You can then re-create the timeseries for each dataframe
out_timeseries_common = out_df_common.groupby([pd.Grouper(freq=str(timestep)+'T', key='time'), 'station_id']).count().unstack('station_id', fill_value=0)
out_timeseries_common.columns = out_timeseries_common.columns.map(str)
out_timeseries_common = out_timeseries_common.reindex(sorted(out_timeseries_common.columns), axis=1)

in_timeseries_common = in_df_common.groupby([pd.Grouper(freq=str(timestep)+'T', key='time'), 'station_id']).count().unstack('station_id', fill_value=0)
in_timeseries_common.columns = in_timeseries_common.columns.map(str)
in_timeseries_common = in_timeseries_common.reindex(sorted(in_timeseries_common.columns), axis=1)

# After this step, out_timeseries_common and in_timeseries_common should have the same time range
print('Out Timeseries range:', common_start_time, "to", common_end_time)
print('In Timeseries range:', common_start_time, "to", common_end_time)

Out Timeseries range: 2023-03-01 00:48:33 to 2023-05-31 23:19:50
In Timeseries range: 2023-03-01 00:48:33 to 2023-05-31 23:19:50


In [72]:
# Set the column names to be the part after the comma and remove the leading '(' and trailing ')'
out_timeseries_common.columns = [col.split(',')[1][:-1].strip(" '") for col in out_timeseries_common.columns]
in_timeseries_common.columns = [col.split(',')[1][:-1].strip(" '") for col in in_timeseries_common.columns]

In [73]:
out_timeseries_common.shape

(17923, 41)

In [74]:
in_timeseries_common.shape

(18023, 41)

In [75]:
# Create a new time index that includes every time point from both DataFrames
common_time_index = out_timeseries_common.index.union(in_timeseries_common.index)

# Reindex both DataFrames to this new time index
out_timeseries_aligned = out_timeseries_common.reindex(common_time_index, fill_value=0)
in_timeseries_aligned = in_timeseries_common.reindex(common_time_index, fill_value=0)

# Now both out_timeseries_aligned and in_timeseries_aligned should have the same length
print('Aligned Out Timeseries length:', len(out_timeseries_aligned))
print('Aligned In Timeseries length:', len(in_timeseries_aligned))

Aligned Out Timeseries length: 20642
Aligned In Timeseries length: 20642


In [76]:
start_time = common_time_index.min()

# Calculate the end time by adding (16992 - 1) * 5 minutes to the start time
end_time = start_time + pd.Timedelta(minutes=5 * (16992 - 1))

# Create a date range with exactly 16992 samples, each 5 minutes apart
new_time_index = pd.date_range(start=start_time, end=end_time, freq='5T')

# Reindex both DataFrames to this new time index, filling missing data with zeros
out_timeseries_aligned = out_timeseries_common.reindex(new_time_index, fill_value=0)
in_timeseries_aligned = in_timeseries_common.reindex(new_time_index, fill_value=0)

# Check the results
print('New Aligned Out Timeseries length:', len(out_timeseries_aligned))
print('New Aligned In Timeseries length:', len(in_timeseries_aligned))

New Aligned Out Timeseries length: 16992
New Aligned In Timeseries length: 16992


In [77]:
out_timeseries_aligned.tail()

,JC002,JC006,JC008,JC009,JC013,JC014,JC018,JC019,JC020,JC022,...,JC081,JC082,JC093,JC094,JC098,JC099,JC102,JC103,JC104,JC105
2023-04-29 00:20:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2023-04-29 00:25:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-04-29 00:30:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-04-29 00:35:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-04-29 00:40:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
in_timeseries_aligned.tail()

,JC002,JC006,JC008,JC009,JC013,JC014,JC018,JC019,JC020,JC022,...,JC081,JC082,JC093,JC094,JC098,JC099,JC102,JC103,JC104,JC105
2023-04-29 00:20:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-04-29 00:25:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-04-29 00:30:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-04-29 00:35:00,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2023-04-29 00:40:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [118]:
in_timeseries_aligned.columns

Index(['JC002', 'JC006', 'JC008', 'JC009', 'JC013', 'JC014', 'JC018', 'JC019',
       'JC020', 'JC022', 'JC023', 'JC024', 'JC027', 'JC032', 'JC038', 'JC051',
       'JC052', 'JC053', 'JC055', 'JC057', 'JC059', 'JC063', 'JC065', 'JC066',
       'JC072', 'JC074', 'JC075', 'JC076', 'JC077', 'JC078', 'JC080', 'JC081',
       'JC082', 'JC093', 'JC094', 'JC098', 'JC099', 'JC102', 'JC103', 'JC104',
       'JC105'],
      dtype='object')

In [79]:
feature1 = out_timeseries_aligned.values
feature2 = in_timeseries_aligned.values

In [88]:
# Stack the features to create a new array of shape (11964, 42, 2)
data = np.stack((feature1, feature2), axis=-1)
data.shape

(16992, 41, 2)

In [91]:
np.savez(r'C:\Users\srush\bike share model\cleaned&merged_data\your_data.npz', feature_data=data)

In [81]:
# Now you can access the second feature for all points
second_feature_data = data[:, :, 1]

# Let's print the shape of the resulting slice
second_feature_data

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

We have 42 nodes, each node (station) has 2 features - number of bikes leaving the station and number of bikes entering the station. each data is collected every 30 mins interval. 

The idea is to sample from the past segments of the data that are predictors for the future segment.

If we want to predict the bike counts in the next hour e.g. (Thursday 8:00 AM) 

We can use :
### 1. The recent segment

We use the last two hours. Based on the traffic from 6:00 to 8:00 AM, we will predict the traffic of 8:00 AM

Intuition : The formation and dispersion of traffic congestions are gradual. So, the just past traffic flows inevitably have influence on the future traffic flows

### 2. The daily-periodic segment

We use the same hour in the last week and the same hour yesterday and the day before. Based on the traffic at 8:00 AM on tuesday and wednesday, we will predict the traffic of 8:00 AM on thursday.

Intuition: Due to the regular daily routine of people, traffic data may show repeated patterns, such as the daily morning peaks. The purpose of the daily-period component is to model the daily periodicity of traffic data.

### 3. The weekly-periodic segment

We use the same hour in the last week and the same hour in the week before the last week. Based on the traffic at 8:00 AM on the thursday in the past two weeks, we will predict the traffic of 8:00 AM

Intuition: Usually, the traffic patterns on Monday have a certain similarity with the traffic patterns on Mondays in history, but may be greatly different from those on weekends. Thus, the weekly-period component is designed to capture the weekly periodic features in traffic data.

In [36]:
def search_data(sequence_length, num_of_depend, label_start_idx,num_for_predict, units, points_per_hour):
    '''
    Parameters
    ----------
    sequence_length: int, length of all history data
    num_of_depend: int,
    label_start_idx: int, the first index of predicting target
    num_for_predict: int, the number of points will be predicted for each sample
    units: int, week: 7 * 24, day: 24, recent(hour): 1
    points_per_hour: int, number of points per hour, depends on data
    Returns
    ----------
    list[(start_idx, end_idx)]
    '''

    if points_per_hour < 0:
        raise ValueError("points_per_hour should be greater than 0!")

    if label_start_idx + num_for_predict > sequence_length:
        return None

    x_idx = []
    for i in range(1, num_of_depend + 1):
        start_idx = label_start_idx - points_per_hour * units * i
        end_idx = start_idx + num_for_predict
        if start_idx >= 0:
            x_idx.append((start_idx, end_idx))
        else:
            return None

    if len(x_idx) != num_of_depend:
        return None

    return x_idx[::-1]

In [85]:
def get_sample_indices(data_sequence, num_of_weeks, num_of_days, num_of_hours, label_start_idx, num_for_predict, points_per_hour=12):
    '''
    Parameters
    ----------
    data_sequence: np.ndarray shape is (sequence_length, num_of_vertices, num_of_features)
    num_of_weeks, num_of_days, num_of_hours: int
    label_start_idx: int, the first index of predicting target
    num_for_predict: int,the number of points will be predicted for each sample
    points_per_hour: int, default 12, number of points per hour
    Returns
    ----------
    week_sample: np.ndarray shape is (num_of_weeks * points_per_hour, num_of_vertices, num_of_features)
    day_sample: np.ndarray shape is (num_of_days * points_per_hour,  num_of_vertices, num_of_features)
    hour_sample: np.ndarray   shape is (num_of_hours * points_per_hour, num_of_vertices, num_of_features)
    target: np.ndarray shape is (num_for_predict, num_of_vertices, num_of_features)
    '''
    week_sample, day_sample, hour_sample = None, None, None
#------------------------------------Ignore
    if label_start_idx + num_for_predict > data_sequence.shape[0]: 
        return week_sample, day_sample, hour_sample, None

    if num_of_weeks > 0:
        week_indices = search_data(data_sequence.shape[0], num_of_weeks, label_start_idx, num_for_predict,7 * 24, points_per_hour)
        if not week_indices:
            return None, None, None, None

        week_sample = np.concatenate([data_sequence[i: j] for i, j in week_indices], axis=0)

    if num_of_days > 0:
        day_indices = search_data(data_sequence.shape[0], num_of_days,  label_start_idx, num_for_predict, 24, points_per_hour)
        if not day_indices:
            return None, None, None, None

        day_sample = np.concatenate([data_sequence[i: j] for i, j in day_indices], axis=0)
#----------------------------------Continue
    if num_of_hours > 0:
        hour_indices = search_data(data_sequence.shape[0], num_of_hours, label_start_idx, num_for_predict, 1, points_per_hour)
        if not hour_indices:
            return None, None, None, None
        hour_sample = np.concatenate([data_sequence[i: j] for i, j in hour_indices], axis=0)
    
    if num_of_hours > 10:
        return 1;
    target = data_sequence[label_start_idx: label_start_idx + num_for_predict]

    return week_sample, day_sample, hour_sample, target

In [106]:
def read_and_generate_dataset(graph_signal_matrix_filename, num_of_weeks, num_of_days, num_of_hours, num_for_predict, points_per_hour=12):
    '''
    Parameters
    ----------
    graph_signal_matrix_filename: str, path of graph signal matrix file
    num_of_weeks, num_of_days, num_of_hours: int
    num_for_predict: int
    points_per_hour: int, default 12, depends on data
    Returns
    ----------
    feature: np.ndarray, shape is (num_of_samples, num_of_depend * points_per_hour, num_of_vertices, num_of_features)
    target: np.ndarray, shape is (num_of_samples, num_of_vertices, num_for_predict)
    '''
    #--------------------------------- Read original data 
    data_seq = np.load(graph_signal_matrix_filename)['feature_data']  # (sequence_length, num_of_vertices, num_of_features) (16992, 307, 3)
    
    #---------------------------------
    all_samples = []
    for idx in range(data_seq.shape[0]):
        sample = get_sample_indices(data_seq, num_of_weeks, num_of_days, num_of_hours, idx, num_for_predict, points_per_hour)
        if ((sample[0] is None) and (sample[1] is None) and (sample[2] is None)):
            continue

        week_sample, day_sample, hour_sample, target = sample #  week_sample, day_sample are None because we are predicting per hour
        #print(target.shape) # hour_sample and target (12, 307, 3)
        sample = []  # [(week_sample),(day_sample),(hour_sample),target,time_sample]

#-------------------------------- Ignore
        if num_of_weeks > 0:
            week_sample = np.expand_dims(week_sample, axis=0).transpose((0, 2, 3, 1))  # (1,N,F,T)
            sample.append(week_sample)

        if num_of_days > 0:
            day_sample = np.expand_dims(day_sample, axis=0).transpose((0, 2, 3, 1))  # (1,N,F,T)
            sample.append(day_sample)
#----------------------------------Continue
        if num_of_hours > 0:
            hour_sample = np.expand_dims(hour_sample, axis=0).transpose((0, 2, 3, 1))  # (1,N,F,T)
            sample.append(hour_sample)

        target = np.expand_dims(target, axis=0).transpose((0, 2, 3, 1))[:, :, 0, :]  # (1,N,T)
        sample.append(target)
        time_sample = np.expand_dims(np.array([idx]), axis=0)  # (1,1)
        sample.append(time_sample)
        all_samples.append(sample)#sampe：[(week_sample),(day_sample),(hour_sample),target,time_sample] = [(1,N,F,Tw),(1,N,F,Td),(1,N,F,Th),(1,N,Tpre),(1,1)]
    
    split_line1 = int(len(all_samples) * 0.6)
    split_line2 = int(len(all_samples) * 0.8)

    training_set = [np.concatenate(i, axis=0)  for i in zip(*all_samples[:split_line1])] #[(B,N,F,Tw),(B,N,F,Td),(B,N,F,Th),(B,N,Tpre),(B,1)]
    validation_set = [np.concatenate(i, axis=0) for i in zip(*all_samples[split_line1: split_line2])]
    testing_set = [np.concatenate(i, axis=0) for i in zip(*all_samples[split_line2:])]

    return training_set, validation_set, testing_set

In [107]:
import numpy as np

graph_signal_matrix_filename = r'C:\\Users\\srush\\bike share model\\cleaned&merged_data\\your_data.npz'
data = np.load(graph_signal_matrix_filename)

# Print all items in the file
print(list(data.keys()))


['feature_data']


In [108]:
graph_signal_matrix_filename = r'C:\Users\srush\bike share model\cleaned&merged_data\your_data.npz'
data = np.load(graph_signal_matrix_filename)
data['feature_data'].shape

(16992, 41, 2)

In [109]:
data = np.load(graph_signal_matrix_filename)
data['feature_data'].shape
num_of_vertices = 41
points_per_hour = 12
num_for_predict = 12
num_of_weeks = 0
num_of_days = 0
num_of_hours = 1

training_set, validation_set, testing_set = read_and_generate_dataset(graph_signal_matrix_filename, 0, 0, num_of_hours, 
                                                                      num_for_predict, points_per_hour=points_per_hour)

### Data Normalization

In [114]:
def normalization(train, val, test):
    '''
    Parameters
    ----------
    train, val, test: np.ndarray (B,N,F,T)
    Returns
    ----------
    stats: dict, two keys: mean and std
    train_norm, val_norm, test_norm: np.ndarray,
                                     shape is the same as original
    '''

    assert train.shape[1:] == val.shape[1:] and val.shape[1:] == test.shape[1:]  # ensure the num of nodes is the same
    mean = train.mean(axis=(0,1,3), keepdims=True)
    std = train.std(axis=(0,1,3), keepdims=True)
    print('mean.shape:',mean.shape)
    print('std.shape:',std.shape)

    def normalize(x):
        return (x - mean) / std

    train_norm = normalize(train)
    val_norm = normalize(val)
    test_norm = normalize(test)

    return {'_mean': mean, '_std': std}, train_norm, val_norm, test_norm

In [115]:
train_x = np.concatenate(training_set[:-2], axis=-1)  # (B,N,F,T')
val_x = np.concatenate(validation_set[:-2], axis=-1)
test_x = np.concatenate(testing_set[:-2], axis=-1)

train_target = training_set[-2]  # (B,N,T)
val_target = validation_set[-2]
test_target = testing_set[-2]

train_timestamp = training_set[-1]  # (B,1)
val_timestamp = validation_set[-1]
test_timestamp = testing_set[-1]

(stats, train_x_norm, val_x_norm, test_x_norm) = normalization(train_x, val_x, test_x)

all_data = {'train': { 'x': train_x_norm, 'target': train_target,'timestamp': train_timestamp},
            'val': {'x': val_x_norm, 'target': val_target, 'timestamp': val_timestamp},
            'test': {'x': test_x_norm, 'target': test_target, 'timestamp': test_timestamp},
            'stats': {'_mean': stats['_mean'], '_std': stats['_std']} }

mean.shape: (1, 1, 2, 1)
std.shape: (1, 1, 2, 1)


In [116]:
print('train x:', all_data['train']['x'].shape)
print('train target:', all_data['train']['target'].shape)
print('train timestamp:', all_data['train']['timestamp'].shape)
print()
print('val x:', all_data['val']['x'].shape)
print('val target:', all_data['val']['target'].shape)
print('val timestamp:', all_data['val']['timestamp'].shape)
print()
print('test x:', all_data['test']['x'].shape)
print('test target:', all_data['test']['target'].shape)
print('test timestamp:', all_data['test']['timestamp'].shape)
print()
print('train data _mean :', all_data['stats']['_mean'].shape, all_data['stats']['_mean'])
print('train data _std :', all_data['stats']['_std'].shape, all_data['stats']['_std'])


train x: (10181, 41, 2, 12)
train target: (10181, 41, 12)
train timestamp: (10181, 1)

val x: (3394, 41, 2, 12)
val target: (3394, 41, 12)
val timestamp: (3394, 1)

test x: (3394, 41, 2, 12)
test target: (3394, 41, 12)
test timestamp: (3394, 1)

train data _mean : (1, 1, 2, 1) [[[[0.0423665 ]
   [0.04235911]]]]
train data _std : (1, 1, 2, 1) [[[[0.22637724]
   [0.22426236]]]]


In [117]:
file = os.path.basename(graph_signal_matrix_filename).split('.')[0]
dirpath = '.'
filename = os.path.join(dirpath, file + '_r' + str(num_of_hours) + '_d' + str(num_of_days) + '_w' + str(num_of_weeks)) + '_astcgn'
print('save file:', filename)
np.savez_compressed(filename,
                train_x=all_data['train']['x'],train_target=all_data['train']['target'],train_timestamp=all_data['train']['timestamp'],
                val_x=all_data['val']['x'], val_target=all_data['val']['target'],val_timestamp=all_data['val']['timestamp'],
                test_x=all_data['test']['x'], test_target=all_data['test']['target'], test_timestamp=all_data['test']['timestamp'],
                mean=all_data['stats']['_mean'], std=all_data['stats']['_std'])

save file: .\your_data_r1_d0_w0_astcgn
